In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\heat_wave\WEIO\expand_WEIO\other_variables\wind_93_19_pacific_warm_pool_area.npz')
print(data.files)  # ['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']

eastward_wind = data['eastward_wind'][:, :64, :48]  # (9861, 41, 201)  #
print(eastward_wind.shape)  # (9861, 41, 201)
northward_wind = data['northward_wind'][:, :64, :48]  # (9861, 41, 201)
wind_vector_divergence = data['wind_vector_divergence'][:, :64, :48]  # (9861, 41, 201)
wind_stress = data['wind_stress'][:, :64, :48]  # (9861, 41, 201)   # 可惜这个是 0
surface_downward_eastward_stress = data['surface_downward_eastward_stress'][:, :64, :48]  # (9861, 41, 201)
surface_downward_northward_stress = data['surface_downward_northward_stress'][:, :64, :48]  # (9861, 41, 201)
wind_stress_curl = data['wind_stress_curl'][:, :64, :48]  # (9861, 41, 201)
wind_stress_divergence = data['wind_stress_divergence'][:, :64, :48]  # (9861, 41, 201)
wind_speed_rms = data['wind_speed_rms'][:, :64, :48]  # (9861, 41, 201)
eastward_wind_rms = data['eastward_wind_rms'][:, :64, :48]  # (9861, 41, 201)
northward_wind_rms = data['northward_wind_rms'][:, :64, :48]  # (9861, 41, 201)
sampling_length = data['sampling_length'][:, :64, :48]  # (9861, 41, 201)
surface_type = data['surface_type'][:, :64, :48]  # (9861, 41, 201)

['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']
(9848, 64, 48)


In [3]:
# 首先 我们先画 JJA季节的贡献图   JJA： 6  7  8 月  即  06/01 - 08/31  当然，会做一个区域平均
# 同时，我们也会对每一年的JJA做一个平均  ---》1993 - 2019年

In [4]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,8,31)) 
index

9738

In [5]:
# JJA   93
eastward_wind_JJA_93 = eastward_wind[151:243,:,:]
northward_wind_JJA_93 = northward_wind[151:243,:,:]
wind_vector_divergence_JJA_93 = wind_vector_divergence[151:243,:,:]
wind_stress_JJA_93 = wind_stress[151:243,:,:]
surface_downward_eastward_stress_JJA_93 = surface_downward_eastward_stress[151:243,:,:]
surface_downward_northward_stress_JJA_93 = surface_downward_northward_stress[151:243,:,:]
wind_stress_curl_JJA_93 = wind_stress_curl[151:243,:,:]
wind_stress_divergence_JJA_93 = wind_stress_divergence[151:243,:,:]
wind_speed_rms_JJA_93 = wind_speed_rms[151:243,:,:]
eastward_wind_rms_JJA_93 = eastward_wind_rms[151:243,:,:]
northward_wind_rms_JJA_93 = northward_wind_rms[151:243,:,:]
sampling_length_JJA_93 = sampling_length[151:243,:,:]
surface_type_JJA_93 = surface_type[151:243,:,:]

In [6]:
# JJA   94
eastward_wind_JJA_94 = eastward_wind[516:608,:,:]
northward_wind_JJA_94 = northward_wind[516:608,:,:]
wind_vector_divergence_JJA_94 = wind_vector_divergence[516:608,:,:]
wind_stress_JJA_94 = wind_stress[516:608,:,:]
surface_downward_eastward_stress_JJA_94 = surface_downward_eastward_stress[516:608,:,:]
surface_downward_northward_stress_JJA_94 = surface_downward_northward_stress[516:608,:,:]
wind_stress_curl_JJA_94 = wind_stress_curl[516:608,:,:]
wind_stress_divergence_JJA_94 = wind_stress_divergence[516:608,:,:]
wind_speed_rms_JJA_94 = wind_speed_rms[516:608,:,:]
eastward_wind_rms_JJA_94 = eastward_wind_rms[516:608,:,:]
northward_wind_rms_JJA_94 = northward_wind_rms[516:608,:,:]
sampling_length_JJA_94 = sampling_length[516:608,:,:]
surface_type_JJA_94 = surface_type[516:608,:,:]

In [7]:
# JJA   95
eastward_wind_JJA_95 = eastward_wind[881:973,:,:]
northward_wind_JJA_95 = northward_wind[881:973,:,:]
wind_vector_divergence_JJA_95 = wind_vector_divergence[881:973,:,:]
wind_stress_JJA_95 = wind_stress[881:973,:,:]
surface_downward_eastward_stress_JJA_95 = surface_downward_eastward_stress[881:973,:,:]
surface_downward_northward_stress_JJA_95 = surface_downward_northward_stress[881:973,:,:]
wind_stress_curl_JJA_95 = wind_stress_curl[881:973,:,:]
wind_stress_divergence_JJA_95 = wind_stress_divergence[881:973,:,:]
wind_speed_rms_JJA_95 = wind_speed_rms[881:973,:,:]
eastward_wind_rms_JJA_95 = eastward_wind_rms[881:973,:,:]
northward_wind_rms_JJA_95 = northward_wind_rms[881:973,:,:]
sampling_length_JJA_95 = sampling_length[881:973,:,:]
surface_type_JJA_95 = surface_type[881:973,:,:]

In [8]:
# JJA   96
time_96_1 = 1247
time_96_2 = 1339
data_1 = time_96_1
data_2 = time_96_2

eastward_wind_JJA_96 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_96 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_96 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_96 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_96 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_96 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_96 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_96 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_96 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_96 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_96 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_96 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_96 = surface_type[data_1 : data_2,:,:]

In [9]:
# JJA   97
time_97_1 = 1612
time_97_2 = 1704
data_1 = time_97_1
data_2 = time_97_2

eastward_wind_JJA_97 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_97 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_97 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_97 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_97 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_97 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_97 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_97 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_97 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_97 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_97 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_97 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_97 = surface_type[data_1 : data_2,:,:]

In [12]:
# JJA   98
time_98_1 = 1977
time_98_2 = 2069
data_1 = time_98_1
data_2 = time_98_2

eastward_wind_JJA_98 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_98 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_98 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_98 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_98 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_98 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_98 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_98 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_98 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_98 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_98 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_98 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_98 = surface_type[data_1 : data_2,:,:]

In [13]:
# JJA   99
time_99_1 = 2342
time_99_2 = 2434
data_1 = time_99_1
data_2 = time_99_2

eastward_wind_JJA_99 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_99 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_99 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_99 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_99 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_99 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_99 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_99 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_99 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_99 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_99 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_99 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_99 = surface_type[data_1 : data_2,:,:]

In [14]:
# JJA   00
time_00_1 = 2708
time_00_2 = 2799
data_1 = time_00_1
data_2 = time_00_2

eastward_wind_JJA_00 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_00 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_00 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_00 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_00 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_00 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_00 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_00 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_00 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_00 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_00 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_00 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_00 = surface_type[data_1 : data_2,:,:]

In [15]:
# JJA   01
time_01_1 = 3073
time_01_2 = 3165
data_1 = time_01_1
data_2 = time_01_2

eastward_wind_JJA_01 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_01 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_01 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_01 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_01 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_01 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_01 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_01 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_01 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_01 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_01 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_01 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_01 = surface_type[data_1 : data_2,:,:]

In [16]:
# JJA   02
time_02_1 = 3438
time_02_2 = 3529
data_1 = time_02_1
data_2 = time_02_2

eastward_wind_JJA_02 = eastward_wind[data_1 : data_2,:,:]
northward_wind_JJA_02 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_JJA_02 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_JJA_02 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_02 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_02 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_JJA_02 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_JJA_02 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_JJA_02 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_JJA_02 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_02 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_JJA_02 = sampling_length[data_1 : data_2,:,:]
surface_type_JJA_02 = surface_type[data_1 : data_2,:,:]

In [17]:
# JJA   03
time_03_1 = 3803
time_03_2 = 3895
data_1 = time_03_1
data_2 = time_03_2

wind_stress_JJA_03 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_03 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_03 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_03 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_03 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_03 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_03 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_03 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_03 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_03 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_03 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_03 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_03 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [18]:
# JJA   04
time_04_1 = 4169
time_04_2 = 4261
data_1 = time_04_1
data_2 = time_04_2

wind_stress_JJA_04 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_04 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_04 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_04 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_04 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_04 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_04 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_04 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_04 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_04 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_04= wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_04 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_04 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [19]:
# JJA   05
time_05_1 = 4534
time_05_2 = 4626
data_1 = time_05_1
data_2 = time_05_2

wind_stress_JJA_05 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_05 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_05 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_05 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_05 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_05 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_05 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_05 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_05 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_05 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_05= wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_05 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_05 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [20]:
# JJA   06
time_06_1 = 4899
time_06_2 = 4990
data_1 = time_06_1
data_2 = time_06_2

wind_stress_JJA_06 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_06 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_06 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_06 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_06 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_06 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_06 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_06 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_06 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_06 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_06 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_06 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_06 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [21]:
# JJA   07
time_07_1 = 5264
time_07_2 = 5356
data_1 = time_07_1
data_2 = time_07_2

wind_stress_JJA_07 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_07 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_07 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_07 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_07 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_07 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_07 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_07 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_07 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_07 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_07 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_07 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_07 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [22]:
# JJA   08
time_08_1 = 5630
time_08_2 = 5722
data_1 = time_08_1
data_2 = time_08_2

wind_stress_JJA_08 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_08 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_08 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_08 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_08 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_08 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_08 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_08 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_08 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_08 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_08 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_08 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_08 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [23]:
# JJA   09
time_09_1 = 5995
time_09_2 = 6087
data_1 = time_09_1
data_2 = time_09_2

wind_stress_JJA_09 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_09 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_09 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_09 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_09 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_09 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_09 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_09 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_09 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_09 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_09 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_09 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_09 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [24]:
# JJA   10
time_10_1 = 6360
time_10_2 = 6452
data_1 = time_10_1
data_2 = time_10_2

wind_stress_JJA_10 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_10 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_10 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_10 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_10 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_10 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_10 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_10 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_10 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_10 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_10 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_10 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_10 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [25]:
# JJA   11
time_11_1 = 6725
time_11_2 = 6817
data_1 = time_11_1
data_2 = time_11_2

wind_stress_JJA_11 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_11 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_11 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_11 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_11 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_11 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_11 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_11 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_11 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_11 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_11 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_11 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_11 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [26]:
# JJA   12
time_12_1 = 7091
time_12_2 = 7183
data_1 = time_12_1
data_2 = time_12_2

wind_stress_JJA_12 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_12 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_12 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_12 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_12 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_12 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_12 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_12 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_12 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_12 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_12 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_12 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_12 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [27]:
# JJA   13
time_13_1 = 7456
time_13_2 = 7548
data_1 = time_13_1
data_2 = time_13_2

wind_stress_JJA_13 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_13 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_13 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_13 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_13 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_13 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_13 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_13 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_13 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_13 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_13 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_13 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_13 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [28]:
# JJA   14
time_14_1 = 7821
time_14_2 = 7913
data_1 = time_14_1
data_2 = time_14_2

wind_stress_JJA_14 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_14 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_14 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_14 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_14 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_14 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_14 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_14 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_14 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_14 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_14 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_14 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_14 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [29]:
# JJA   15
time_15_1 = 8186
time_15_2 = 8278
data_1 = time_15_1
data_2 = time_15_2

wind_stress_JJA_15 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_15 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_15 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_15 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_15 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_15 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_15 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_15 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_15 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_15 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_15 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_15 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_15 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [30]:
# JJA   16
time_16_1 = 8552
time_16_2 = 8644
data_1 = time_16_1
data_2 = time_16_2

wind_stress_JJA_16 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_16 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_16 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_16 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_16 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_16 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_16 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_16 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_16 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_16 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_16 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_16 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_16 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [31]:
# JJA   17
time_17_1 = 8917
time_17_2 = 9009
data_1 = time_17_1
data_2 = time_17_2

wind_stress_JJA_17 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_17 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_17 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_17 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_17 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_17 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_17 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_17 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_17 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_17 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_17 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_17 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_17 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [34]:
# JJA   18
time_18_1 = 9280
time_18_2 = 9369
data_1 = time_18_1
data_2 = time_18_2

wind_stress_JJA_18 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_18 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_18 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_18 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_18 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_18 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_18 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_18 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_18 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_18 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_18 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_18 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_18 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [35]:
# JJA   19
time_19_1 = 9634
time_19_2 = 9726 
data_1 = time_19_1 
data_2 = time_19_2 

wind_stress_JJA_19 = wind_stress[data_1 : data_2,:,:]
surface_type_JJA_19 = surface_type[data_1 : data_2,:,:]
eastward_wind_JJA_19 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_JJA_19 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_JJA_19 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_JJA_19 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_JJA_19 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_JJA_19 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_JJA_19 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_JJA_19 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_JJA_19 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_JJA_19 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_JJA_19 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [52]:
wind_stress_JJA = np.concatenate((wind_stress_JJA_93, wind_stress_JJA_94, wind_stress_JJA_95,
                                 wind_stress_JJA_96, wind_stress_JJA_97, wind_stress_JJA_98, wind_stress_JJA_99,
                                 wind_stress_JJA_00, wind_stress_JJA_01, wind_stress_JJA_02, wind_stress_JJA_03,
                                 wind_stress_JJA_04, wind_stress_JJA_05, wind_stress_JJA_06, wind_stress_JJA_07,
                                 wind_stress_JJA_08, wind_stress_JJA_09, wind_stress_JJA_10, wind_stress_JJA_11,
                                 wind_stress_JJA_12, wind_stress_JJA_13, wind_stress_JJA_14, wind_stress_JJA_15,
                                 wind_stress_JJA_16, wind_stress_JJA_17, wind_stress_JJA_18, wind_stress_JJA_19), axis = 0)

In [54]:
surface_type_JJA = np.concatenate((surface_type_JJA_93, surface_type_JJA_94, surface_type_JJA_95,
                                  surface_type_JJA_96, surface_type_JJA_97, surface_type_JJA_98, surface_type_JJA_99,
                                  surface_type_JJA_00, surface_type_JJA_01, surface_type_JJA_02, surface_type_JJA_03,
                                  surface_type_JJA_04, surface_type_JJA_05, surface_type_JJA_06, surface_type_JJA_07,
                                  surface_type_JJA_08, surface_type_JJA_09, surface_type_JJA_10, surface_type_JJA_11,
                                  surface_type_JJA_12, surface_type_JJA_13, surface_type_JJA_14, surface_type_JJA_15,
                                  surface_type_JJA_16, surface_type_JJA_17, surface_type_JJA_18, surface_type_JJA_19), axis = 0)

In [56]:
eastward_wind_JJA = np.concatenate((eastward_wind_JJA_93, eastward_wind_JJA_94, eastward_wind_JJA_95,
                                   eastward_wind_JJA_96, eastward_wind_JJA_97, eastward_wind_JJA_98,
                                   eastward_wind_JJA_99, eastward_wind_JJA_00, eastward_wind_JJA_01,
                                   eastward_wind_JJA_02, eastward_wind_JJA_03, eastward_wind_JJA_04,
                                   eastward_wind_JJA_05, eastward_wind_JJA_06, eastward_wind_JJA_07,
                                   eastward_wind_JJA_08, eastward_wind_JJA_09, eastward_wind_JJA_10,
                                   eastward_wind_JJA_11, eastward_wind_JJA_12, eastward_wind_JJA_13,
                                   eastward_wind_JJA_14, eastward_wind_JJA_15, eastward_wind_JJA_16,
                                   eastward_wind_JJA_17, eastward_wind_JJA_18, eastward_wind_JJA_19), axis = 0)

In [58]:
wind_stress_curl_JJA = np.concatenate((wind_stress_curl_JJA_93, wind_stress_curl_JJA_94, wind_stress_curl_JJA_95,
                                      wind_stress_curl_JJA_96, wind_stress_curl_JJA_97, wind_stress_curl_JJA_98, 
                                      wind_stress_curl_JJA_99, wind_stress_curl_JJA_00, wind_stress_curl_JJA_01,
                                      wind_stress_curl_JJA_02, wind_stress_curl_JJA_03, wind_stress_curl_JJA_04, 
                                      wind_stress_curl_JJA_05, wind_stress_curl_JJA_06, wind_stress_curl_JJA_07,
                                      wind_stress_curl_JJA_08, wind_stress_curl_JJA_09, wind_stress_curl_JJA_10,
                                      wind_stress_curl_JJA_11, wind_stress_curl_JJA_12, wind_stress_curl_JJA_13,
                                      wind_stress_curl_JJA_14, wind_stress_curl_JJA_15, wind_stress_curl_JJA_16,
                                      wind_stress_curl_JJA_17, wind_stress_curl_JJA_18, wind_stress_curl_JJA_19), axis = 0)

In [59]:
northward_wind_JJA = np.concatenate((northward_wind_JJA_93, northward_wind_JJA_94, northward_wind_JJA_95,
                                    northward_wind_JJA_96, northward_wind_JJA_97, northward_wind_JJA_98,
                                    northward_wind_JJA_99, northward_wind_JJA_00, northward_wind_JJA_01,
                                    northward_wind_JJA_02, northward_wind_JJA_03, northward_wind_JJA_04, 
                                    northward_wind_JJA_05, northward_wind_JJA_06, northward_wind_JJA_07,
                                    northward_wind_JJA_08, northward_wind_JJA_09, northward_wind_JJA_10,
                                    northward_wind_JJA_11, northward_wind_JJA_12, northward_wind_JJA_13,
                                    northward_wind_JJA_14, northward_wind_JJA_15, northward_wind_JJA_16,
                                    northward_wind_JJA_17, northward_wind_JJA_18, northward_wind_JJA_19), axis = 0)

In [60]:
wind_speed_rms_JJA = np.concatenate((wind_speed_rms_JJA_93, wind_speed_rms_JJA_94, wind_speed_rms_JJA_95,
                                    wind_speed_rms_JJA_96, wind_speed_rms_JJA_97, wind_speed_rms_JJA_98,
                                    wind_speed_rms_JJA_99, wind_speed_rms_JJA_00, wind_speed_rms_JJA_01,
                                    wind_speed_rms_JJA_02, wind_speed_rms_JJA_03, wind_speed_rms_JJA_04,
                                    wind_speed_rms_JJA_05, wind_speed_rms_JJA_06, wind_speed_rms_JJA_07,
                                    wind_speed_rms_JJA_08, wind_speed_rms_JJA_09, wind_speed_rms_JJA_10,
                                    wind_speed_rms_JJA_11, wind_speed_rms_JJA_12, wind_speed_rms_JJA_13,
                                    wind_speed_rms_JJA_14, wind_speed_rms_JJA_15, wind_speed_rms_JJA_16,
                                    wind_speed_rms_JJA_17, wind_speed_rms_JJA_18, wind_speed_rms_JJA_19), axis = 0)

In [61]:
sampling_length_JJA = np.concatenate((sampling_length_JJA_93, sampling_length_JJA_94, sampling_length_JJA_95,
                                     sampling_length_JJA_96, sampling_length_JJA_97, sampling_length_JJA_98,
                                     sampling_length_JJA_99, sampling_length_JJA_00, sampling_length_JJA_01,
                                     sampling_length_JJA_02, sampling_length_JJA_03, sampling_length_JJA_04,
                                     sampling_length_JJA_05, sampling_length_JJA_06, sampling_length_JJA_07,
                                     sampling_length_JJA_08, sampling_length_JJA_09, sampling_length_JJA_10,
                                     sampling_length_JJA_11, sampling_length_JJA_12, sampling_length_JJA_13,
                                     sampling_length_JJA_14, sampling_length_JJA_15, sampling_length_JJA_16,
                                     sampling_length_JJA_17, sampling_length_JJA_18, sampling_length_JJA_19), axis = 0)

In [62]:
eastward_wind_rms_JJA = np.concatenate((eastward_wind_rms_JJA_93, eastward_wind_rms_JJA_94, eastward_wind_rms_JJA_95,
                                       eastward_wind_rms_JJA_96, eastward_wind_rms_JJA_97, eastward_wind_rms_JJA_98,
                                       eastward_wind_rms_JJA_99, eastward_wind_rms_JJA_00, eastward_wind_rms_JJA_01,
                                       eastward_wind_rms_JJA_02, eastward_wind_rms_JJA_03, eastward_wind_rms_JJA_04,
                                       eastward_wind_rms_JJA_05, eastward_wind_rms_JJA_06, eastward_wind_rms_JJA_07,
                                       eastward_wind_rms_JJA_08, eastward_wind_rms_JJA_09, eastward_wind_rms_JJA_10,
                                       eastward_wind_rms_JJA_11, eastward_wind_rms_JJA_12, eastward_wind_rms_JJA_13,
                                       eastward_wind_rms_JJA_14, eastward_wind_rms_JJA_15, eastward_wind_rms_JJA_16,
                                       eastward_wind_rms_JJA_17, eastward_wind_rms_JJA_18, eastward_wind_rms_JJA_19), axis = 0)

In [63]:
northward_wind_rms_JJA = np.concatenate((northward_wind_rms_JJA_93, northward_wind_rms_JJA_94,
                                        northward_wind_rms_JJA_95, northward_wind_rms_JJA_96,northward_wind_rms_JJA_97,
                                        northward_wind_rms_JJA_98, northward_wind_rms_JJA_99, northward_wind_rms_JJA_00,
                                        northward_wind_rms_JJA_01, northward_wind_rms_JJA_02, northward_wind_rms_JJA_03,
                                        northward_wind_rms_JJA_04, northward_wind_rms_JJA_05, northward_wind_rms_JJA_06,
                                        northward_wind_rms_JJA_07, northward_wind_rms_JJA_08, northward_wind_rms_JJA_09,
                                        northward_wind_rms_JJA_10, northward_wind_rms_JJA_11, northward_wind_rms_JJA_12,
                                        northward_wind_rms_JJA_13, northward_wind_rms_JJA_14, northward_wind_rms_JJA_15,
                                        northward_wind_rms_JJA_16, northward_wind_rms_JJA_17, northward_wind_rms_JJA_18,
                                        northward_wind_rms_JJA_19), axis = 0)

In [64]:
wind_stress_divergence_JJA = np.concatenate((wind_stress_divergence_JJA_93, wind_stress_divergence_JJA_94,
                                            wind_stress_divergence_JJA_95, wind_stress_divergence_JJA_96, 
                                            wind_stress_divergence_JJA_97, wind_stress_divergence_JJA_98,
                                            wind_stress_divergence_JJA_99, wind_stress_divergence_JJA_00,
                                            wind_stress_divergence_JJA_01, wind_stress_divergence_JJA_02,
                                            wind_stress_divergence_JJA_03, wind_stress_divergence_JJA_04,
                                            wind_stress_divergence_JJA_05, wind_stress_divergence_JJA_06, 
                                            wind_stress_divergence_JJA_07, wind_stress_divergence_JJA_08,
                                            wind_stress_divergence_JJA_09, wind_stress_divergence_JJA_10,
                                            wind_stress_divergence_JJA_11, wind_stress_divergence_JJA_12,
                                            wind_stress_divergence_JJA_13, wind_stress_divergence_JJA_14,
                                            wind_stress_divergence_JJA_15, wind_stress_divergence_JJA_16,
                                            wind_stress_divergence_JJA_17, wind_stress_divergence_JJA_18, wind_stress_divergence_JJA_19), axis = 0)

In [66]:
wind_vector_divergence_JJA = np.concatenate((wind_vector_divergence_JJA_93, wind_vector_divergence_JJA_94,
                                            wind_vector_divergence_JJA_95, wind_vector_divergence_JJA_96,
                                            wind_vector_divergence_JJA_97, wind_vector_divergence_JJA_98,
                                            wind_vector_divergence_JJA_99, wind_vector_divergence_JJA_00,
                                            wind_vector_divergence_JJA_01, wind_vector_divergence_JJA_02,
                                            wind_vector_divergence_JJA_03, wind_vector_divergence_JJA_04,
                                            wind_vector_divergence_JJA_05, wind_vector_divergence_JJA_06,
                                            wind_vector_divergence_JJA_07, wind_vector_divergence_JJA_08,
                                            wind_vector_divergence_JJA_09, wind_vector_divergence_JJA_10,
                                            wind_vector_divergence_JJA_11, wind_vector_divergence_JJA_12,
                                            wind_vector_divergence_JJA_13, wind_vector_divergence_JJA_14,
                                            wind_vector_divergence_JJA_15, wind_vector_divergence_JJA_16,
                                            wind_vector_divergence_JJA_17, wind_vector_divergence_JJA_18, wind_vector_divergence_JJA_19), axis = 0)

In [67]:
surface_downward_eastward_stress_JJA = np.concatenate((surface_downward_eastward_stress_JJA_93,
                                                      surface_downward_eastward_stress_JJA_94, surface_downward_eastward_stress_JJA_95,
                                                      surface_downward_eastward_stress_JJA_96, surface_downward_eastward_stress_JJA_97,
                                                      surface_downward_eastward_stress_JJA_98, surface_downward_eastward_stress_JJA_99,
                                                      surface_downward_eastward_stress_JJA_00, surface_downward_eastward_stress_JJA_01,
                                                      surface_downward_eastward_stress_JJA_02, surface_downward_eastward_stress_JJA_03,
                                                      surface_downward_eastward_stress_JJA_04, surface_downward_eastward_stress_JJA_05,
                                                      surface_downward_eastward_stress_JJA_06, surface_downward_eastward_stress_JJA_07,
                                                      surface_downward_eastward_stress_JJA_08, surface_downward_eastward_stress_JJA_09,
                                                      surface_downward_eastward_stress_JJA_10, surface_downward_eastward_stress_JJA_11,
                                                      surface_downward_eastward_stress_JJA_12, surface_downward_eastward_stress_JJA_13,
                                                      surface_downward_eastward_stress_JJA_14, surface_downward_eastward_stress_JJA_15,
                                                      surface_downward_eastward_stress_JJA_16, surface_downward_eastward_stress_JJA_17,
                                                      surface_downward_eastward_stress_JJA_18, surface_downward_eastward_stress_JJA_19), axis = 0)

In [68]:
surface_downward_northward_stress_JJA = np.concatenate((surface_downward_northward_stress_JJA_93,
                                                       surface_downward_northward_stress_JJA_94, surface_downward_northward_stress_JJA_95,
                                                       surface_downward_northward_stress_JJA_96, surface_downward_northward_stress_JJA_97,
                                                       surface_downward_northward_stress_JJA_98, surface_downward_northward_stress_JJA_99,
                                                       surface_downward_northward_stress_JJA_00, surface_downward_northward_stress_JJA_01,
                                                       surface_downward_northward_stress_JJA_02, surface_downward_northward_stress_JJA_03,
                                                       surface_downward_northward_stress_JJA_04, surface_downward_northward_stress_JJA_05,
                                                       surface_downward_northward_stress_JJA_06, surface_downward_northward_stress_JJA_07,
                                                       surface_downward_northward_stress_JJA_08, surface_downward_northward_stress_JJA_09,
                                                       surface_downward_northward_stress_JJA_10, surface_downward_northward_stress_JJA_11,
                                                       surface_downward_northward_stress_JJA_12, surface_downward_northward_stress_JJA_13,
                                                       surface_downward_northward_stress_JJA_14, surface_downward_northward_stress_JJA_15,
                                                       surface_downward_northward_stress_JJA_16, surface_downward_northward_stress_JJA_17,
                                                       surface_downward_northward_stress_JJA_18, surface_downward_northward_stress_JJA_19), axis = 0)

In [ ]:
np.savez(r'D:\heat_wave\WEIO\expand_WEIO\other_variables\last\wind_93_19_WEIO_area_JJA.npz',
        wind_stress_JJA = wind_stress_JJA, surface_type_JJA = surface_type_JJA, eastward_wind_JJA = eastward_wind_JJA,
        wind_stress_curl_JJA = wind_stress_curl_JJA, northward_wind_JJA = northward_wind_JJA, 
        wind_speed_rms_JJA = wind_speed_rms_JJA, sampling_length_JJA = sampling_length_JJA,
        eastward_wind_rms_JJA = eastward_wind_rms_JJA, northward_wind_rms_JJA = northward_wind_rms_JJA,
        )